In [ ]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#import graphviz
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import kaggle_xgb
import calc_ndcg
#from sklearn.grid_search import ParameterGrid
import pickle
import dataEngr
reload(dataEngr)

Read in just the testing and training data.

In [ ]:
xgbInput = dataEngr.clfInput()
xgbInput.get_sessionsFtr()
xgbInput.users_ftrEng()
xgbInput.one_hot()
#xgbInput.binarize_targets()
xgbInput.split_data(update_trainDf=True)

In [ ]:
%run -i custom_paramsearch.py

In [ ]:
condition = 'dac_year == 2014 & dac_month == 4 & action_counts != -1'
valid_mask = xgbInput.trainDf.index.isin(xgbInput.trainDf.query(condition).index)

In [ ]:
def cv_sessions(xgbInput):
    """Cross-validation on a subset of training data which meets a condition"""
    
    condition = 'dac_year == 2014 & action_counts != -1'
    idx = np.nonzero(xgbInput.trainDf.index.isin(xgbInput.trainDf.query(condition).index))
    idx = idx[0]
    np.random.shuffle(idx[0])
    n = len(idx)
    
    for i in range(0, kfolds):
        idx[0]
        
def cv_bydate(xgbInput):
    """Select folds for cross validation as all cases that occurred in a given with month 
    in 2014, with sessions data.
    Only cases in 2014 have sessions data and the last test case is on June 30.
    """
    
    for i in range(4,7):
        condition = 'dac_year == 2014 & dac_month == @i & action_counts != -1'
        valid_mask = xgbInput.trainDf.index.isin(xgbInput.trainDf.query(condition).index)
        valid_indx = np.where(valid_mask)[0]
        train_indx = np.where(~valid_mask)[0]
        yield train_indx, valid_indx

In [ ]:
a = pd.read_pickle('cv_results/actions_e20/validate_err.p')
a = a.astype(float)
a['mean'] = a.mean(axis = 1)

In [ ]:
with open('../xgbmodels/actions2_e13_96n.p') as f:
    bst = pickle.load(f)

In [ ]:
xgbInput.allDf.columns[xgbInput.allDf.columns.str.contains('book')]

In [ ]:
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, xgbInput.allDf.columns[int(feat[1:])], score) for feat, score in features]